In [1]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib as plt
import itertools

In [49]:
def mmn_queueing_jsq(n,arrival_rate,service_rate,simulation_time=60,simuseed=math.nan):
    '''
    Simulating the mmn queueing system using the method of JSQ 
    (namely join the shortest queue method), the user needs to provide
    the number of servers (n),arrival rate (lambda) and service rate (mu),
    in case of n >= 2, if mu is provided as a scaler, then all servers
    share the same mu, if mu is provided as a vector, then all servers may have different mu,
    the simulation time is by default 60 units, and for reproducibility purposes, the user
    can define simulation seed, by default the seed value is not set.
    
    n:               the number of servers, a positive finite integer
    arrival_rate:    exponential rate of arrival
    service_rate:    exponential rate of service time
    simulation_time: the simulation time, a positive finite number
    simuseed:        the simulation seed number
    
    The output will xxx
    '''
    
    if math.isnan(simuseed) == True:
        pass
    else:
        random.seed(simuseed)

    if not type(n) is int:
        raise TypeError("n needs to be integer")
    if n < 0:
        raise Exception("n needs to be positive")
    
    lamb =  arrival_rate
    
    if isinstance(service_rate, list):
        if len(service_rate) == n:
            mu = service_rate
        elif (n > 1 and len(service_rate) == 1):
            mu = n*service_rate
        else:
            raise Exception("Unmatched cardinality for service rate and n")
    elif isinstance(service_rate, int):
        if n == 1:
            mu = [service_rate]
        elif n > 1:
            mu = n*[service_rate]
    else:
        raise Exception("Please use int or list type for service rate")
    
    
    # Simulating the event calendar
    table_col = np.arange(n)*4+3
    avail_col = np.arange(n)*4+4
    queue_col = np.arange(n)*4+5
    total_col = np.arange(n)*4+6
    compare_col = np.insert(avail_col,0,[1,2])
    
    event_calendar = np.zeros([2,3+4*n])
    event_calendar[-1,1] = random.expovariate(lamb)
    event_calendar[-1,2] = simulation_time
    event_calendar[-1,avail_col] = math.nan
    
    while event_calendar[-1,0] < simulation_time:
        event_calendar = np.vstack([event_calendar,event_calendar[-1,:]])
        event_calendar[-1,0] = min(event_calendar[-2,compare_col])
        
        # If the next event is an arrival
        if event_calendar[-1,0] == event_calendar[-2,1]:
            event_calendar[-1,1] = event_calendar[-1,0]+random.expovariate(lamb)
            server_assignment = random.choice(np.where(event_calendar[-1,total_col] == \
                                   min(event_calendar[-1,total_col]))[0])
            
            # If the server is not occupied
            if event_calendar[-1,table_col[server_assignment]] == 0:
                event_calendar[-1,table_col[server_assignment]] += 1
                event_calendar[-1,avail_col[server_assignment]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_assignment])
                event_calendar[-1,total_col[server_assignment]] += 1
            
            # Else if the server is occupied
            elif event_calendar[-1,table_col[server_assignment]] == 1:
                event_calendar[-1,queue_col[server_assignment]] += 1
                event_calendar[-1,total_col[server_assignment]] += 1
            
        # Else if the next event is the simulation termination
        elif event_calendar[-1,0] == simulation_time:
            pass
        
        # Else if the next event is a service completion
        else:
            server_completion = np.where(event_calendar[-2,avail_col] == event_calendar[-1,0])[0][0]
            
            # If there is no queue behind
            if event_calendar[-1,queue_col[server_completion]] == 0:
                event_calendar[-1,table_col[server_completion]] -= 1
                event_calendar[-1,avail_col[server_completion]] = math.nan
                event_calendar[-1,total_col[server_completion]] -= 1
            
            # Else if there is a queue behind
            elif event_calendar[-1,queue_col[server_completion]] > 0:
                event_calendar[-1,avail_col[server_completion]] = event_calendar[-1,0] + \
                                                        random.expovariate(mu[server_completion])
                event_calendar[-1,queue_col[server_completion]] -= 1
                event_calendar[-1,total_col[server_completion]] -= 1
    
    event_calendar = pd.DataFrame(event_calendar).tail(-1)
    event_calendar.columns = ['Time','Next Customer','Finish Time'] + \
                    list(itertools.chain.from_iterable([[f"Server{i}", \
                    f"Server{i} Available Time",f"Server{i} Queue",f"Server{i} Total"] for i in range(1,n+1)]))
    event_calendar["Grand Total"] = event_calendar.iloc[:,total_col].sum(axis=1)
    
    # Concluding performance measures
    # Time dynamic
    time1 = event_calendar.iloc[:-1,0].to_numpy()
    time2 = event_calendar.iloc[1:,0].to_numpy()
    time3 = time2 - time1
    
    # Calculate the utilization for each server
    utiliz_server = event_calendar.iloc[:-1,table_col].to_numpy()
    utiliz_data = ((time3 @ utiliz_server)/simulation_time).round(3)
    utiliz = pd.DataFrame(utiliz_data).transpose()
    utiliz.columns = [f"Server{i} Utilization" for i in range(1,n+1)]
    
    # Calculating the average number in the systems for each server and grand total
    ans_server = event_calendar.iloc[:-1,np.append(total_col,event_calendar.shape[1]-1)].to_numpy()
    ans_data = ((time3 @ ans_server)/simulation_time).round(3)
    ans = pd.DataFrame(ans_data).transpose()
    ans.columns = [f"Server{i} Average # in system" for i in range(1,n+1)]+["Grand Average # in system"]
    
    # Calculating the average queue length
    
    
    
    
    
    # Ploting the queue length time graph
    
    
    
    
    
    return ans#event_calendar, utiliz


In [11]:
mmn_queueing_jsq(2,10,[3,10],simulation_time=20,simuseed=8)[0].iloc[:,[0,-1]]

,Time,Grand Total
1,0.000000,0.0
2,0.025710,1.0
3,0.097235,0.0
4,0.353506,1.0
5,0.357982,2.0
...,...,...
370,19.897019,5.0
371,19.952403,4.0
372,19.977297,5.0
373,19.988873,4.0


In [50]:
mmn_queueing_jsq(2,10,[3,10],simulation_time=20,simuseed=8)#.iloc[:,[0,-1]]

,Server1 Average # in system,Server2 Average # in system,Grand Average # in system
0,2.46,1.757,4.217


In [126]:
mmn_queueing_jsq(4,60,[3,10,19,5],simulation_time=70,simuseed=8)

,Server1 Utilization,Server2 Utilization,Server3 Utilization,Server4 Utilization
0,0.999,0.998,0.997,0.997
